In [ ]:
import pandas as pd
from getlyrics import az, lcom_artist_searcher, lcom_lyrics
from queue import Queue
import threading, json, requests, time, random, os, re, textdistance
from bs4 import BeautifulSoup


In [3]:
tracks_df = pd.read_pickle("./tracks.pkl") 
tracks_df.size

for index, row in tracks_df.iterrows():
    print(tracks_df.loc[index])
    break
    
    

title                       New Bottega
artists    [Torren Foot, Azealia Banks]
lyrics                              NaN
Name: 0, dtype: object


In [ ]:
# Initalise queue and threading
q = Queue()
thread = threading.Thread(target=az, args=())

az_df = pd.DataFrame(columns = ['title','artists', 'lyrics'])

In [ ]:
def get_func(url, sleep_range):
    sleep_range = [5,10]
    header = {'User-Agent': 'Lyrics language checker'}
    r = requests.get(url, headers = header)
    if r.status_code != 200:
        print(f"\n{r.status_code} from {url}")
        return 0
    else:
        time.sleep(random.uniform(sleep_range[0], sleep_range[1]))
        return r
    
    #============================================================#
def az_artist_searcher(artist):
    encoded_artist = re.sub(r"\s", "+", artist)
    #search for an artist on AZ lyrics
    url = f"https://search.azlyrics.com/search.php?q={encoded_artist}"
    r = get_func(url, (20, 30))
    soup = BeautifulSoup(r.text, 'html.parser')
    
    if not re.search("Artist results", r.text):
        return 0
    
    link_list = []
    for item in soup.select('div'):
        if item.attrs == {'class': ['panel']}:
            if item.contents[1].find('b').get_text() == "Artist results:":
                for link in item.find_all('a'):
                    if "https" in str(link):
                        link_list.append(link.get('href'))
                        
    #picks the result which most closely matches the queried artist (or the only result if there is only 1)
    if len(link_list) == 1:
        link = link_list[0]
        r = get_func(link, (20, 30))
        soup = BeautifulSoup(r.text, 'html.parser') 
        
    else:
        for i, link in enumerate(link_list):
            l = link.split("/")
            art = l[len(l)-1][:-5]
            jaro = textdistance.jaro_winkler(re.sub(r"\s", "", artist), art)
            link_list[i] = (jaro, link)
        link_list.sort()

        url = link_list[-1][1]
        r = get_func(url, (20, 30))
        soup = BeautifulSoup(r.text, 'html.parser')
    
    #finds and returns the full list of songs present on AZlyrics with corresponding link to lyrics page
    songs_dict = {}
    for link in soup.find_all('a'):
        l = link.get('href')
        if l:
            if "/lyrics/" in l:
                if ".." in l:
                    l  = "https://www.azlyrics.com/" + l[2:]
                    songs_dict.update({link.get_text().lower(): l})
                else:
                    songs_dict.update({link.get_text().lower(): l})

    
    return songs_dict



def az_song_searcher(song):
    artists = song['artists']
    title = song['title']
    ### copy some logic from the other for artists loop
    for artist in artists:
        song = "+".join(song)
        song = re.sub(r"\s", "+", song)
        r = get_func(f"https://search.azlyrics.com/search.php?q={song}&x=62ea26e9d8f4fb60ae9f8e35bc3ee3feb516419c1b5e2f1d62061d6238d0acb3", (10, 20))
    
        if r == 0:
            return 0
        if not re.search("Song results", r.text):
            with open ("lyric.txt", "w") as f:
                f.write(r.text)
            return 0
        else:
            soup = BeautifulSoup(r.text, 'html.parser') 
            link_list = []
            for item in soup.select('div'):
                if item.attrs == {'class': ['panel']}:
                    if item.contents[1].find('b').get_text() == "Song results:":
                        for link in item.find_all('a'):
                            if "https" in str(link):
                                link_list.append(link.get('href'))
    
        for link in link_list:
            ll = link.split("/")
            print(ll)
            print(re.sub(r"\s", "", artist), ll[4])
            art = textdistance.jaro_winkler(re.sub(r"\s", "", artist), ll[4])
            tit = textdistance.jaro_winkler(re.sub(r"\s", "", title), ll[5][:-5])
            if art and tit > 0.8:
                return get_az_lyrics(link)
            else:
                return 0

def get_az_lyrics(url):
    r = get_func(url, (10, 20))
    if r == 0:
        return 0
    soup = BeautifulSoup(r.text, 'html.parser')
    l = soup.find_all("div", attrs={"class": None, "id": None})
    if not l:
        return 0
    elif l:
        l = [x.getText() for x in l]
        return l
    
def az():
    while q.empty() == False:
        song = q.get()
        lyrics = az_song_searcher(song)
        if lyrics == 0:
            pass
        else:
            checklang(song, lyrics[0])
            
        
        qlist=[item for item in q.queue]
            
    #============================================================#

    
def lcom_searcher(artist):
    artist = re.sub(r"\s", "+", artist)
    r = get_func(f"https://www.lyrics.com/serp.php?st={artist}", (2, 4))
    if r == 0:
        return 0
    artist_found = False
    soup = BeautifulSoup(r.text, 'html.parser')
    for item in soup.find_all("tr"):
        for link in item.find_all('a'):
            l = link.get_text()
            if textdistance.jaro_winkler(link.get_text().lower(), artist) >= 0.9:
                if "sub-artist" not in link.get('href'):
                    artist_link = f"https://www.lyrics.com/{link.get('href')}"
                    adata = get_func(artist_link, (2,4))
                    if r == 0:
                        return 0
                    artist_found = True
                    return adata
                    
    
    if artist_found == False:
        return 0
    
def lcom_lyrics(url):
    ldata = get_func(url, (2, 4))
    if ldata == 0:
        return 0
    soup = BeautifulSoup(ldata.text, 'html.parser')
    for item in soup.select("pre"):
        lyrics = item.get_text()
        return(lyrics)
    
def lcom_artist_searcher(artist):
    adata = lcom_searcher(artist)
    if adata == 0:
        return 0
    
    else:
        song_list = []
        soup = BeautifulSoup(adata.text, 'html.parser')
        for link in soup.find_all('a'):
            try:
                if "/lyric/" in link.get('href'):
                    song_list.append([link.get_text().lower(), f"https://www.lyrics.com/{link.get('href')}"])
            except TypeError:
                pass
        
        return song_list           

In [7]:


for index, row in tracks_df.iterrows():
    artists = row['artists']
    title = row['title']

    flag = False
    # Search for each listed artist of song supplied by spotyify on lyrics.com 
    for artist in artists:
        
        #retrieve list of songs by that artist
        song_list = lcom_artist_searcher(artist)
        
        
        # if that artist not found on lyrics.com, first search for the next one (if there is one), then pass it on to azlyrics
        if song_list == 0:
            # if the flag is not set, it means that the first artist was not found on lyrics.com
            # so we set the flag to True and continue to the next artist for the same song
            if flag == False:
                flag = True
                continue

            # put into the queue for az function to process
            q.put(row)
            
            if not thread.is_alive():
                thread = threading.Thread(target=az, args=())
                thread.start()
            
        #otherwise if the artist is found on lyrics.com, we try to match the song title with the retrieved song list
        else:
            flag = False
            for song in song_list:
                #attempts to match to a song from the retrieved song list 
                if textdistance.jaro_winkler(title, song[0]) >= 0.9:
                    match = True

                    lyrics = lcom_lyrics(song[1])
                    #If the page is empty of lyrics put in AZlyrics queue
                    if lyrics == None:
                        q.put(row)
                        break
                    #If another exception is raised (HTTP error), put it in queue
                    elif lyrics == 0:
                        q.put(row)
                        break
                    else:
                        tracks_df.loc[index]['lyrics'] = lyrics
                    break

            #if no match for the song is found on lyrics.com, query azlyrics instead
            if match == False:
                q.put(row)
                if not thread.is_alive():
                    thread = threading.Thread(target=az, args=())
                    thread.start()

                

Exception in thread Thread-11 (az):
Traceback (most recent call last):
  File "C:\Users\charl\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "c:\Users\charl\OneDrive\Desktop\lyrics_project\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\charl\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\charl\OneDrive\Desktop\lyrics_project\getlyrics.py", line 164, in az
    while q.empty() == False:
          ^
NameError: name 'q' is not defined


KeyboardInterrupt: 